## Plot maps with solution (dam porftolios) for different power generation targets
### T. Janus
### Designed: 06.02.2024
### Modified: 12.11.2024

In [ ]:
import pathlib
import copy
import json
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

from lib.notebook13 import (
    SeabornFig2Grid, plot_mya_reservoirs, make_radar_plots, plot_rivord_histograms,
    plot_elev_dist, plot_histograms
)

In [ ]:
import matplotlib
from matplotlib.lines import Line2D
from shapely.geometry import Point
import matplotlib.gridspec as gridspec
def add_reservoir_contours(
        data: pd.DataFrame,
        ax: matplotlib.axes.Axes,
        lon_field: str = "coordinates_1",
        lat_field: str = "coordinates_0",
        res_label_field: str | None = None,
        title: str | None = None,
        title_font_size: int = 14,
        marker_size: int | str = 40,
        marker_size_multiplier: float = 1.0,
        cmap = None,
        plot_legends = (False, False, False),
        offset: float = 0.0,
        legend_y_coords = (1.0, 0.2, -0.05),
        marker_constant = 1.00,
        **kwargs):
    """Plots the maps with dams"""
    crs={'init':'epsg:4326'}
    geometry=[Point(xy) for xy in zip(data[lon_field], data[lat_field])]
    data_gdf=gpd.GeoDataFrame(data,crs=crs, geometry=geometry)
    # Divide dams into storage and RoR
    data_gdf_ror = data_gdf[data_gdf['hp_type_reem'] == 'ror']
    data_gdf_sto = data_gdf[data_gdf['hp_type_reem'] == 'sto']
    
    data_gdf_sto_hp = data_gdf[
        (data_gdf['hp_type_reem'] == 'sto') & (data_gdf['type_y'] == 'hydroelectric')]
    data_gdf_sto_mp = data_gdf[
        (data_gdf['hp_type_reem'] == 'sto') & (data_gdf['type_y'] == 'multipurpose')]
    
    if isinstance(marker_size, str):
        marker_size_ror = np.sqrt(data_gdf_ror[marker_size].astype(float))
        marker_size_sto = np.sqrt(data_gdf_sto[marker_size].astype(float))
        marker_size_sto_hp = np.sqrt(data_gdf_sto_hp[marker_size].astype(float))
        marker_size_sto_mp = np.sqrt(data_gdf_sto_mp[marker_size].astype(float))
    else:
        marker_size_ror = marker_size
        marker_size_sto = marker_size
        marker_size_sto_hp = marker_size
        marker_size_sto_mp = marker_size
        
    p1 = data_gdf_ror.plot(
        kind='geo', ax=ax, cmap=cmap, 
        column=None, 
        facecolor="none",            # No fill color for transparency
        #color="white",
        #hatch='///',                 # Hatch pattern (slanted lines)
        markersize=marker_size_ror*marker_size_multiplier*marker_constant,
        marker = "^", k=10,
        alpha=0.6,
        linewidth=0.5, edgecolor='red', categorical = True, 
        legend_kwds={
            'loc':'upper right', 
            'bbox_to_anchor': (1.17+offset, legend_y_coords[0]),
            'markerscale':0.5, 
            'title_fontsize':'medium', 
            'frameon':False,
            'fontsize':"small"},
        legend=plot_legends[0],
        **kwargs)
    p2 = data_gdf_sto_hp.plot(
        kind = 'geo', ax=ax, 
        column=None, 
        cmap=cmap, 
        alpha=0.6,
        facecolor="none",            # No fill color for transparency
        #hatch='///',                 # Hatch pattern (slanted lines)
        markersize=marker_size_sto_hp*marker_size_multiplier*marker_constant, 
        marker = "o", k = 10,
        linewidth=0.5, edgecolor='red', categorical = True,        
        **kwargs)
    p3 = data_gdf_sto_mp.plot(
        kind = 'geo', ax=ax, 
        column=None, 
        cmap=cmap, 
        aspect=1,
        alpha=0.6,
        facecolor="none",            # No fill color for transparency
        #hatch='///',                 # Hatch pattern (slanted lines)
        markersize=marker_size_sto_mp*marker_size_multiplier*marker_constant, 
        marker = "s", k = 10,
        linewidth=0.5, edgecolor='red', categorical = True, **kwargs)
    
    #You can use different 'cmaps' such as jet, plasm,magma, infereno,cividis, binary...(I simply chose cividis)
    ax.set_yticks([])
    ax.set_xticks([])
    x_spacing = 3
    y_spacing = 3
    #ax.set_xlim(data_gdf.total_bounds[0] - x_spacing/2, data_gdf.total_bounds[2] + x_spacing/2)
    #ax.set_ylim(data_gdf.total_bounds[1] - y_spacing/2, data_gdf.total_bounds[3] + y_spacing/2)
    if title is not None:
        ax.set_title(title, fontsize=title_font_size, loc='left', y=1.0, pad=-18)
    # Remove axes
    for pos in ('top', 'right', 'bottom', 'left'):
        ax.spines[pos].set_visible(False)
    # Label the reservoirs
    if res_label_field is not None:
        for x, y, label in zip(data['coordinates_1'], data['coordinates_0'], data[res_label_field]):
            ax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points', fontsize=5, alpha=0.7)

    return p1, p2, p3

def plot_mya_reservoirs(
        data: pd.DataFrame,
        ax: matplotlib.axes.Axes,
        column_name: str,
        lon_field: str = "coordinates_1",
        lat_field: str = "coordinates_0",
        rivers_shp: str | None = r'bin/gis_layers/mya_rivers.shp',
        outline_shp: str | None = r'bin/gis_layers/myanmar_outline/Myanmar_outline.shp',
        res_label_field: str | None = None,
        title: str | None = None,
        title_font_size: int = 14,
        marker_size: int | str = 40,
        marker_size_multiplier: float = 1.0,
        cmap = 'tab10',
        plot_legends = (False, False, False),
        offset: float = 0.0,
        legend_y_coords = (1.0, 0.2, -0.05),
        contour: bool = False,
        **kwargs):
    """Plots the maps with dams"""
    crs={'init':'epsg:4326'}
    geometry=[Point(xy) for xy in zip(data[lon_field], data[lat_field])]
    data_gdf=gpd.GeoDataFrame(data,crs=crs, geometry=geometry)
    if rivers_shp is not None:
        rivers = gpd.read_file(rivers_shp, crs=crs)
        rivers.plot(ax=ax, edgecolor = 'k', linewidth=0.3, alpha = 0.5)
    if outline_shp is not None:
        outline = gpd.read_file(outline_shp)
        outline.plot(ax=ax, facecolor='Grey', edgecolor='k',alpha=0.05,linewidth=1,cmap="cividis")
        outline.plot(ax=ax, facecolor='none', edgecolor='k',alpha=1,linewidth=0.2,cmap="cividis")
    # Divide dams into storage and RoR
    data_gdf_ror = data_gdf[data_gdf['hp_type_reem'] == 'ror']
    data_gdf_sto = data_gdf[data_gdf['hp_type_reem'] == 'sto']
    
    data_gdf_sto_hp = data_gdf[(data_gdf['hp_type_reem'] == 'sto') & (data_gdf['type_y'] == 'hydroelectric')]
    data_gdf_sto_mp = data_gdf[(data_gdf['hp_type_reem'] == 'sto') & (data_gdf['type_y'] == 'multipurpose')]
    
    if isinstance(marker_size, str):
        marker_size_ror = np.sqrt(data_gdf_ror[marker_size].astype(float))
        marker_size_sto = np.sqrt(data_gdf_sto[marker_size].astype(float))
        marker_size_sto_hp = np.sqrt(data_gdf_sto_hp[marker_size].astype(float))
        marker_size_sto_mp = np.sqrt(data_gdf_sto_mp[marker_size].astype(float))
    else:
        marker_size_ror = marker_size
        marker_size_sto = marker_size
        marker_size_sto_hp = marker_size
        marker_size_sto_mp = marker_size
    
    if contour:
        skwargs = {
            'alpha': 0.6,
            'facecolor': "none",
            'linewidth': 0.75,
            'edgecolor': 'firebrick',
            'column': column_name
        }
    else:
        skwargs = {
            'alpha': 0.45,
            'edgecolor': 'k',
            'linewidth': 0.5,
            'column': column_name
        }
    
    #kwargs.update(skwargs)
    
    p1 = data_gdf_ror.plot(
        kind='geo', ax=ax, cmap=cmap,
        markersize=marker_size_ror*marker_size_multiplier,
        marker = "^", k=10,
        categorical = True, 
        legend_kwds={
            'loc':'upper right', 
            'bbox_to_anchor': (1.17+offset, legend_y_coords[0]),
            'markerscale':0.5, 
            'title_fontsize':'medium', 
            'frameon':False,
            'fontsize':"small"},
        legend=plot_legends[0],
        **skwargs,
        **kwargs)
    
    p2 = data_gdf_sto_hp.plot(
        kind = 'geo', ax=ax, cmap=cmap, 
        markersize=marker_size_sto_hp*marker_size_multiplier, 
        marker = "o", k = 10,
        categorical = True,  
        **skwargs,
        **kwargs)
    p3 = data_gdf_sto_mp.plot(
        kind = 'geo', ax=ax, aspect=1, cmap=cmap,
        markersize=marker_size_sto_mp*marker_size_multiplier, 
        marker = "s", k = 10,
        categorical = True, **skwargs, **kwargs)
    
    if plot_legends[0]:
        leg1 = p1.get_legend()
        leg1._legend_box.align = "left"
        custom_labels = ["0.0-3.1", "3.1-20", "20-40", "40-60", "60-100", "100-400", "400-800", "800-5000"]
        if leg1:
            leg1.set_title("GHG Emission\nIntensity\n[gCO$_{2,eq}$/kWh]")
        if leg1 and custom_labels:
            new_legtxt = custom_labels
            for ix,eb in enumerate(leg1.get_texts()):
                eb.set_text(new_legtxt[ix])
        ax.add_artist(leg1)
    
    if plot_legends[1]:
        legend_elements = [
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='Sto HP', markerfacecolor='none', markersize=10),
            Line2D([0], [0], marker='s', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color="none",
                   label='Sto Multipurpose', markerfacecolor='none', markersize=10),
            Line2D([0], [0], marker='^', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='RoR', markerfacecolor='none', markersize=10)]
        ax.legend(handles=legend_elements, **{
                'loc':'upper right', 
                'bbox_to_anchor': (1.2+offset, legend_y_coords[1]),
                'markerscale':1, 
                'title_fontsize':'medium', 
                'frameon':False,
                'fontsize':"small"})
        leg2 = ax.get_legend()
        leg2._legend_box.align = "left"
        leg2.set_title("HP Type")
        ax.add_artist(leg2)
    if plot_legends[2]:
        legend_elements = [
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='10MW', markerfacecolor='none', markersize=4),
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color="none",
                   label='100MW', markerfacecolor='none', markersize=10),
            Line2D([0], [0], marker='o', markeredgecolor='k', markeredgewidth=0.2, alpha=1,
                   color='none',
                   label='1000MW', markerfacecolor='none', markersize=20)]
        ax.legend(handles=legend_elements, **{
                'loc':'upper right', 
                'bbox_to_anchor': (1.2+offset, legend_y_coords[2]),
                'markerscale':1.00, 
                'title_fontsize':'medium', 
                'frameon':False,
                'fontsize':'small'})
        leg3 = ax.get_legend()
        leg3._legend_box.align = "left"
        leg3.set_title("Mean Power Output")
    
    #You can use different 'cmaps' such as jet, plasm,magma, infereno,cividis, binary...(I simply chose cividis)
    ax.set_yticks([])
    ax.set_xticks([])
    x_spacing = 3
    y_spacing = 3
    #ax.set_xlim(data_gdf.total_bounds[0] - x_spacing/2, data_gdf.total_bounds[2] + x_spacing/2)
    #ax.set_ylim(data_gdf.total_bounds[1] - y_spacing/2, data_gdf.total_bounds[3] + y_spacing/2)
    if title is not None:
        ax.set_title(title, fontsize=title_font_size, loc='left', y=1.0, pad=-18)
    # Remove axes
    for pos in ('top', 'right', 'bottom', 'left'):
        ax.spines[pos].set_visible(False)
    # Label the reservoirs
    if res_label_field is not None:
        for x, y, label in zip(data['coordinates_1'], data['coordinates_0'], data[res_label_field]):
            ax.annotate(label, xy=(x,y), xytext=(4,4), textcoords='offset points', fontsize=5, alpha=0.7)

    return p1, p2, p3

In [ ]:
# File paths
all_dams_shp_file_path = pathlib.Path(
    "bin/gis_layers/ifc_database/all_dams_replaced_refactored.shp")
dam_data_plotting_path = pathlib.Path(
    "intermediate/dams_for_plotting_moo.csv")
optim_scenarios_5obj_path = pathlib.Path(
    'intermediate/optim_scenarios_5obj.json')
optim_scenarios_5obj_soued_path = pathlib.Path(
    'intermediate/optim_scenarios_5obj_soued.json')
all_hp_path = pathlib.Path('outputs/moo/all_hp.csv')

In [ ]:
all_hp = pd.read_csv(all_hp_path, index_col = 'ifc_id')

In [ ]:
all_hp.head(2)

In [ ]:
all_hp.loc[236]

### Load data

In [ ]:
# 1. Load elevation data and basin data from ifc database
ifcdb_df = gpd.read_file(all_dams_shp_file_path)\
    .loc[:, ['IFC_ID', 'DAM_NAME', 'Basin', 'Sub-Basin', 'Sub-Basi_1', 'RIV_ORD', 'FSL (m)']]\
    .set_index('IFC_ID')
# Set missing FSL values from Pywr model data
ifcdb_df.at[8, 'FSL (m)'] = 313 #400
ifcdb_df.at[10, 'FSL (m)'] = 323.0 #428
ifcdb_df.at[120, 'FSL (m)'] = 178

# 2. Load outputs file with MOO results
outputs_file = dam_data_plotting_path
output_df = pd.read_csv(outputs_file, index_col=None)
# output_df has duplicated ifc ids. Therefore, assign ifc_id of Mali dam to 236
output_df.loc[output_df['name'] == "Mali", "ifc_id"] = 236
output_df.set_index("ifc_id", inplace=True)
# Define bin edges for land loss
bins = [0, 3.1, 20, 40, 60, 100, 400, 800, 5000]
# Define labels for the bins
#labels = ['0-400 km2', '400-800 km2', '800-1200 km2', '1200-1600 km2']
output_df['GHG intensity cat [gCO2,eq/kWh]'] = pd.cut(
    output_df['GHG intensity [gCO2,eq/kWh]'], bins=bins)
output_df['GHG intensity Soued cat [gCO2,eq/kWh]'] = pd.cut(
    output_df['GHG intensity Soued [gCO2,eq/kWh]'], bins=bins)

output_df['GHG intensity HP cat [gCO2,eq/kWh]'] = pd.cut(
    output_df['GHG intensity HP [gCO2,eq/kWh]'], bins=bins)
output_df['GHG intensity Soued HP cat [gCO2,eq/kWh]'] = pd.cut(
    output_df['GHG intensity Soued HP [gCO2,eq/kWh]'], bins=bins)

# 3. Load file with selected dams per optimization scenario
with open(pathlib.Path('intermediate/optim_scenarios.json'), 'r') as file:
    optim_scenarios = json.load(file)
    
with open(optim_scenarios_5obj_path, 'r') as file:
    optim_scenarios_5obj = json.load(file)
    
with open(optim_scenarios_5obj_soued_path, 'r') as file:
    optim_scenarios_5obj_soued = json.load(file)
    
df_combined = ifcdb_df

output_df_riv_ord = output_df.reset_index().merge(
    ifcdb_df.reset_index()[['IFC_ID', 'RIV_ORD']], left_on="ifc_id", right_on="IFC_ID",
    how="left").set_index("ifc_id")

# Create a map of scenarios: optimization with g-res GHG emissions and GHG emissions from emission factors
options = {
    '5obj': {
        'scenario': optim_scenarios_5obj
    },
    '5obj_soued': {
        'scenario': optim_scenarios_5obj_soued
    }
}

In [ ]:
from typing import List
def list_nan_indices(df: pd.DataFrame, col_name: str) -> List[int]:
    return df[df[col_name].isna()].index.tolist()

In [ ]:
# Assert that there are no intensity values that are NaN
assert not list_nan_indices(df=output_df, col_name='GHG intensity cat [gCO2,eq/kWh]')

In [ ]:
def compare_scenarios(
        sc1, 
        sc2, 
        sc1_name: str = "5obj", 
        sc2_name: str = "5obj_soued"):
    for (sc1, dams1), (sc2, dams2) in zip(sc1.items(), sc2.items()):
        if not sc1 == sc2:
            continue
        print(f"Scenario {sc1}")
        dam_difference_sc1_sc2 = set(dams1) - set(dams2)
        if dam_difference_sc1_sc2:
            dams_sc1_not_in_sc2 = ", ".join(list(map(str, dam_difference_sc1_sc2)))
        else:
            dams_sc1_not_in_sc2 = "None"
        dam_difference_sc2_sc1 = set(dams2) - set(dams1)
        if dam_difference_sc2_sc1:
            dams_sc2_not_in_sc1 = ", ".join(list(map(str, dam_difference_sc2_sc1)))
        else:
            dams_sc2_not_in_sc1 = "None"
        print(f"Dams in {sc1_name} not featured in {sc2_name} - {dams_sc1_not_in_sc2}")
        print(f"Dams in {sc2_name} not featured in {sc1_name} - {dams_sc2_not_in_sc1}")

In [ ]:
# Correct scenario Ib
optim_scenarios_5obj['Ib'].append(150)
optim_scenarios_5obj_soued['Ib'].append(165)

In [ ]:
compare_scenarios(optim_scenarios_5obj, optim_scenarios_5obj_soued)

In [ ]:
output_df.head(2)

In [ ]:
print("Columns\n----------\n"+"\n".join(output_df.columns))

## Create data for the radar plot

In [ ]:
df_radar = copy.deepcopy(
    output_df_riv_ord[[
        'forest_area_loss_km2', 'crop_area_loss_km2', 'Area, km2', 'GHG, tCO2eq/yr', 
        'GHG Soued, tCO2eq/yr', 'Firm Power Ratio, [%]', 'HP Production [GWh/year]', 
        'RIV_ORD']])
# Metric 1
df_radar['Loss of forest, [%]'] = \
    df_radar['forest_area_loss_km2'] / (df_radar['forest_area_loss_km2']).sum() * 100
# Metric 2
df_radar['Loss of agricultural land, [%]'] = \
    df_radar['crop_area_loss_km2'] / (df_radar['crop_area_loss_km2']).sum() * 100
# Metric 3
df_radar['GHG emissions, [%]'] = \
    df_radar['GHG, tCO2eq/yr'] / df_radar['GHG, tCO2eq/yr'].sum() * 100
# Alternative Metric 3 using GHG emission information of Soued
df_radar['GHG emissions (IPCC), [%]'] = \
    df_radar['GHG Soued, tCO2eq/yr'] / df_radar['GHG Soued, tCO2eq/yr'].sum() * 100

# Metric 4
# Firm Power Ratio, [%]
# Metric 5
# Percentage of total HP Production
df_radar['HP Production, [%]'] = \
    df_radar['HP Production [GWh/year]'] / df_radar['HP Production [GWh/year]'].sum() * 100
# Create two dataframes - one for plotting data with emissions according to Gres and the other one with
# emissions according to IPCC
df_radar = df_radar.drop(columns = [
    'forest_area_loss_km2', 'crop_area_loss_km2', 'Area, km2', 'GHG, tCO2eq/yr', 
    'HP Production [GWh/year]', 'GHG Soued, tCO2eq/yr'
])

In [ ]:
df_radar.head(2)

### Plot the maps

In [ ]:
include_statistics = False
if include_statistics:
    plot_type = 'with_statistics'
else:
    plot_type: str = 'without_statistics'
# Select scenario
optim_scenario: str = '5obj'
_optim_scenarios = options[optim_scenario]['scenario']
_optim_scenarios_5obj = options['5obj']['scenario']
_optim_scenarios_5obj_soued = options['5obj_soued']['scenario']
# Create a plot with both scenarios as an option
total_plot: bool = True

In [ ]:
output_df

In [ ]:
add_soued_plots = True
    
title_fontsize = 18

fig, axs = plt.subplots(
    2,3, figsize = (9,8), 
    gridspec_kw=dict(
            left=.02, bottom=.02, right=.98, top=.98, wspace=.02, hspace=.0
        ),
    layout="constrained")
scenario_id_to_latex = {
    "Ib" : "$I_b$",
    "Inb" : "$I_{nb}$",
    "IIb" : "$II_b$",
    "IInb" : "$II_{nb}$",
    "IIIb" : "$III_b$",
    "IIInb" : "$III_{nb}$"}
column_name = 'GHG intensity cat [gCO2,eq/kWh]'
vmin = 0
vmax = 500

# reorder the optim_scenarios_dictionary
desired_order = ['Ib', 'IIb', 'IIIb', 'Inb', 'IInb', 'IIInb']
# Create a new dictionary with keys in the desired order
reordered_scenarios_5obj = {key: _optim_scenarios_5obj[key] for key in desired_order}
reordered_scenarios_5obj_soued = {key: _optim_scenarios_5obj_soued[key] for key in desired_order}

for ix, (scenario_id, dams) in enumerate(reordered_scenarios_5obj.items()):
    """ """
    dams_soued = reordered_scenarios_5obj_soued[scenario_id]
    data = output_df.loc[dams]
    data_soued = output_df.loc[dams_soued]

    #print(f"Scenario: {scenario_id}, Plotting dams {dams}")
    #print(set(dams) - set(dams_soued))
    #print(set(dams_soued) - set(dams))
    # Add plots
    if ix == 3:
        plt.subplots_adjust(right=0.9) 
        p1, p2, p3 = plot_mya_reservoirs(
            data = data, 
            rivers_shp = r'bin/gis_layers/mya_rivers.shp',
            outline_shp = r'bin/gis_layers/myanmar_outline/Myanmar_outline.shp',
            column_name = column_name,
            ax=axs.flat[ix], 
            title=scenario_id_to_latex[scenario_id],
            marker_size = 'HP Production [GWh/year]', 
            marker_size_multiplier=2.5, 
            title_font_size=title_fontsize, 
            cmap='RdYlGn_r', 
            vmin=0, 
            vmax = 9, 
            plot_legends = (True, False, False),
            legend_y_coords = (1.0, 1.0, 1.0),
            offset=1.1)
        if add_soued_plots:
            plt.subplots_adjust(right=0.9) 
            p1, p2, p3 = plot_mya_reservoirs(
                data = data_soued, 
                rivers_shp = None,
                outline_shp = None,
                column_name = None,
                ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', 
                marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, 
                cmap=None, 
                vmin=0, 
                vmax = 9, 
                plot_legends = (True, False, False),
                legend_y_coords = (1.0, 1.0, 1.0),
                contour = True,
                offset=1.1)
    elif ix == 5:
        p1, p2, p3 = plot_mya_reservoirs(
            data = data, 
            rivers_shp = r'bin/gis_layers/mya_rivers.shp',
            outline_shp = r'bin/gis_layers/myanmar_outline/Myanmar_outline.shp',
            column_name = column_name, 
            ax=axs.flat[ix], 
            title=scenario_id_to_latex[scenario_id],
            marker_size = 'HP Production [GWh/year]', 
            marker_size_multiplier=2.5, 
            title_font_size=title_fontsize, 
            cmap='RdYlGn_r', 
            vmin=0, 
            vmax = 9, 
            plot_legends = (True, True, True),
            legend_y_coords = (1.0, 1.0, 0.7),
            offset=1.1)
        if add_soued_plots:
            p1, p2, p3 = plot_mya_reservoirs(
                data = data_soued, 
                rivers_shp = None,
                outline_shp = None,
                column_name = None, 
                ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', 
                marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, 
                cmap=None, 
                vmin=0, 
                vmax = 9,
                contour = True,
                plot_legends=(False, False, False),
                offset=1.1)
    else:
        p1, p2, p3 = plot_mya_reservoirs(
            data = data, 
            rivers_shp = r'bin/gis_layers/mya_rivers.shp',
            outline_shp = r'bin/gis_layers/myanmar_outline/Myanmar_outline.shp',
            column_name = column_name, 
            ax=axs.flat[ix], 
            title=scenario_id_to_latex[scenario_id],
            marker_size = 'HP Production [GWh/year]', 
            marker_size_multiplier=2.5, 
            title_font_size=title_fontsize, 
            cmap='RdYlGn_r', 
            vmin=0, 
            vmax = 9,
            plot_legends=(False, False, False))
        if add_soued_plots:
            p1, p2, p3 = plot_mya_reservoirs(
                data = data_soued, 
                rivers_shp = None,
                outline_shp = None,
                column_name = None, 
                ax=axs.flat[ix], 
                title=scenario_id_to_latex[scenario_id],
                marker_size = 'HP Production [GWh/year]', 
                marker_size_multiplier=2.5, 
                title_font_size=title_fontsize, 
                cmap=None, 
                vmin=0, 
                vmax = 9,
                contour = True,
                plot_legends=(False, False, False))

# 'HP Production [GWh/year]'
fig.tight_layout(pad=0.0)
fig.show()
fig.savefig(
    pathlib.Path(
        f'outputs/figures/moo/GHG_emissions_optim_maps_composite_{optim_scenario}.png'), 
    transparent=True, 
    bbox_inches='tight',
    dpi=600)
fig.savefig(
    pathlib.Path(
        f'outputs/figures/moo/GHG_emissions_optim_maps_composite_{optim_scenario}.svg'),
    bbox_inches='tight')

In [ ]:
df_radar

In [ ]:
from math import pi
from lib.notebook13 import calc_composite_metrics_radar_df
def make_radar_plots(
        axs, 
        df_radar: pd.DataFrame, 
        optim_scenarios: pd.DataFrame,
        **kwargs) -> None:
    """ """
    optim_scenarios_keys = list(optim_scenarios.keys())
    optim_scenarios_alt = kwargs.get('optim_scenarios_alt', None)
    
    if optim_scenarios_alt:
        # Process additional data containing alternative scenarios for comparison
        # By convention alternative scenarios will be plotted with thin dashed lines
        optim_scenarios_keys_alt = list(optim_scenarios_alt.keys())
    
    for ix, (scenario_id, dams) in enumerate(optim_scenarios.items()):
        """ """
        if ix * 2 + 1 > len(optim_scenarios_keys):
            break
        key1 = optim_scenarios_keys[ix * 2]
        key2 = optim_scenarios_keys[ix * 2 + 1]
        dams1 = optim_scenarios[key1]
        dams2 = optim_scenarios[key2]
        
        marker_size = 10
        # Calculate the number of categories in the radar plot
        categories=list(df_radar)[:]
        N = len(categories)
        # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
        angles = [n / float(N) * 2 * pi for n in range(N)]
        angles += angles[:1] 
        
        scenarios = ("I", "II", "III")
        ax = axs[ix]

        handles = [
            Line2D([], [],  lw=0.4, c=color, marker="o", markersize=marker_size/6, label=species)
            for species, color in zip(["Built", "Not Built"], ['tab:blue', 'tab:orange'])
        ]

        if ix == 2:
            legend = ax.legend(
                handles=handles,
                prop={'size': 8},
                loc=(1, 0),       # bottom-right
                labelspacing=0.5, # add space between labels
                frameon=False     # don't put a frame
            )
            
        data1 = df_radar.loc[dams1]
        data1_summary = calc_composite_metrics_radar_df(data1)
        data2 = df_radar.loc[dams2]
        data2_summary = calc_composite_metrics_radar_df(data2)
        
        ax.set_theta_offset(pi / 2)
        ax.set_theta_direction(-1)
        
        ax.set_yticks([])
        ax.set_xticks([])
        ax.yaxis.grid(True)
        ax.xaxis.grid(True)
        
        categories_abbr = list(data1_summary.index)
        ax.set_xticks(angles[:-1], categories_abbr, size=8)

        # First series
        values1=data1_summary.values.flatten().tolist()
        values1 += values1[:1]
        # Plot the first series
        ax.plot(angles, values1, linewidth=0.75, linestyle='solid', label="Built", alpha=0.5)
        ax.fill(angles, values1, 'tab:blue', alpha=0.1)
        ax.scatter(angles, values1, s=marker_size, zorder=10)
        # Second series
        values2=data2_summary.values.flatten().tolist()
        values2 += values2[:1]
        ax.plot(angles, values2, linewidth=0.75, linestyle='solid', label="Not Built", alpha=0.5)
        ax.fill(angles, values2, 'tab:orange', alpha=0.1)
        ax.scatter(angles, values2, s=marker_size, zorder=10)

        max_value = max(values1 + values2)
        #upper_limit = min([value for value in ytick_values if value > max_value])
        upper_limit = max_value
        ax.set_ylim(0, upper_limit)

        ax.set_xticks(angles[:-1])
        ax.tick_params(pad=3, size=7)
        
        ax.spines["start"].set_color("none")
        
        HANGLES = np.linspace(0, 2 * np.pi)
        
        H0 = np.zeros(len(HANGLES))
        H1 = np.ones(len(HANGLES)) * 20
        H2 = np.ones(len(HANGLES)) * 40
        H3 = np.ones(len(HANGLES)) * 60
        H4 = np.ones(len(HANGLES)) * 80
        H5 = np.ones(len(HANGLES)) * 100
        
        if upper_limit == 80:
            Hs = {0: H0, 40: H2, 80: H4}
        elif upper_limit == 100:
            Hs = {0: H0, 20: H1, 60: H3, 100:H5}
        else:
            Hs = {0: H0, 20: H1, 40: H2, 60: H3, 100:H5}

        for key, hangle in Hs.items():
            if key > upper_limit:
                break
            ax.plot(HANGLES, hangle, lw=0.8, ls=(0, (5, 6)), c='Grey')

        text_angle = -37.1
        PAD = 2
        
        ax.set_title(
            scenarios[ix], fontsize=14, loc='left', y=1.2, pad=-2, fontdict={'fontstyle': 'italic'})
        ax.tick_params(axis='x', labelsize=9)
        
        for level in Hs:
            if level == 0:
                continue
            if level > upper_limit:
                break
            text = f"{level}%"
            ax.text(text_angle, level + PAD, text, size=6, alpha=0.7)
            
    if optim_scenarios_alt:
        for ix, (scenario_id, dams) in enumerate(optim_scenarios_alt.items()):
            """ """
            if ix * 2 + 1 > len(optim_scenarios_keys_alt):
                break
            categories=list(df_radar)[:]
            N = len(categories)
            # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
            angles = [n / float(N) * 2 * pi for n in range(N)]
            angles += angles[:1] 
            ax = axs[ix]
            key1_alt = optim_scenarios_keys_alt[ix * 2]
            key2_alt = optim_scenarios_keys_alt[ix * 2 + 1]
            dams1_alt = optim_scenarios_alt[key1_alt]
            dams2_alt = optim_scenarios_alt[key2_alt]
            
            data1_alt = df_radar.loc[dams1_alt]
            data1_summary_alt = calc_composite_metrics_radar_df(data1_alt)
            data2_alt = df_radar.loc[dams2_alt]
            data2_summary_alt = calc_composite_metrics_radar_df(data2_alt)
            
            # First series
            values1_alt=data1_summary_alt.values.flatten().tolist()
            values1_alt += values1_alt[:1]
            # Plot the first series
            ax.plot(
                angles, values1_alt, marker='+',
                ms=6,
                color='dimgray', linewidth=0.15,
                linestyle='none',
                markeredgewidth=0.35,
                label="Built", alpha=1, zorder=20)
            #ax.scatter(angles, values1, 'k+', zorder=10)
            # Second series
            values2_alt=data2_summary_alt.values.flatten().tolist()
            values2_alt += values2_alt[:1]
            ax.plot(
                angles, values2_alt, marker='+',
                color='dimgray', linewidth=0.15,
                markeredgewidth=0.35,
                ms=6,
                linestyle='none', 
                label="Not Built", alpha=1, zorder=20)
            #ax.scatter(angles, values2, 'k+', zorder=10)
            
    return

In [ ]:
## Plot statistics on a separate plot
title_fontsize = 18
fig, axs = plt.subplots(
    3,3, figsize = (6.5,4), 
    gridspec_kw=dict(
            left=.02, bottom=.02, right=.98, top=.98, wspace=.02, hspace=.0
        ),
    layout="constrained")
for ax in axs.flat[:]:
    ax.remove()  
# Remove the remaining (unused) axes 
gs = axs[0,0].get_gridspec()
gs.update(wspace=1, hspace=1)
statistics_grid = gs[:, :]
nrows = 3
ncols = 3

gs00 = statistics_grid.subgridspec(nrows, ncols, hspace=0.45,wspace=0.6)

polar_axs = []
cart_axs = []
cart_axs2 = []
for nrow in range(nrows):
    for ncol in range(ncols):
        if nrow == 0:
            polar_axs.append(fig.add_subplot(gs00[nrow, ncol], projection='polar'))
        elif nrow == 1:
            cart_axs.append(fig.add_subplot(gs00[nrow, ncol]))
        else:
            cart_axs2.append(fig.add_subplot(gs00[nrow, ncol]))

make_radar_plots(
    polar_axs, df_radar.drop(
        columns=["RIV_ORD", "GHG emissions (IPCC), [%]"]
    ), 
    optim_scenarios_5obj, 
    optim_scenarios_alt = optim_scenarios_5obj_soued)
#plot_histograms(cart_axs, output_df_riv_ord, optim_scenarios)
plot_elev_dist(cart_axs, output_df_riv_ord, _optim_scenarios_5obj, df_combined)
plot_rivord_histograms(cart_axs2, output_df_riv_ord, _optim_scenarios_5obj)


# 'HP Production [GWh/year]'
fig.tight_layout(pad=0.50)
fig.show()
fig.savefig(
    pathlib.Path(f'outputs/figures/moo/GHG_emissions_stats_{optim_scenario}.png'), 
    transparent=True, dpi=600, bbox_inches='tight')
fig.savefig(
    pathlib.Path(
        f'outputs/figures/moo/GHG_emissions_stats_{optim_scenario}.svg'),
    bbox_inches='tight')
fig.savefig(
    pathlib.Path(
        f'outputs/figures/moo/GHG_emissions_stats_{optim_scenario}.pdf'),
    bbox_inches='tight')

### Unused code